# Extract `member_ID`s from `community_page` and scrape member pages

- A big 'community page'-- a Q&A page for members with questions, scrolled down multiple times-- was acquired to extract member ids.  >1000 ids were obtained.  
- scraper.Scrape is used to scrape member pages from the allrecipes site.  
- member pages are stored in mongodb collection `member_pages`


In [1]:
import pymongo 
from bs4 import BeautifulSoup as BS
import pickle
import boto3
import os 
AWS_KEY = os.environ['AWS_ACCESS_KEY']
AWS_SECRET = os.environ['AWS_SECRET_ACCESS_KEY'] 

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
allrecipes_db = client.allrecipes

In [3]:
allrecipes_db.collection_names()

[u'community_link_pages',
 u'recipes',
 u'member_pages',
 u'members',
 u'webpages',
 u'healthy_recipes',
 u'about',
 u'community_page']

In [4]:
community_coll = allrecipes_db.community_page
communitypage = community_coll.find_one()['page']

In [17]:
member_coll = allrecipes_db.members
member_pages_coll = allrecipes_db.member_pages

In [6]:
soup = BS(communitypage, 'html.parser')

In [7]:
# Why?  
# with open("community_page.html", "w") as f: 
#     pickle.dump(communitypage, f)

In [8]:
# Extract the link 
members = [(a.attrs['href'], a.text) for a in soup.select('div.ap-avatar a')]

In [9]:
# All links pertaining to members in community page
len(members)

6120

In [10]:
memberIDs = [a[0].split('/')[4] for a in members]

In [11]:
# Exclude duplicates: 
memberIDs = list(set(memberIDs))

len(memberIDs)

1340

In [18]:
# Verify number of members in members_coll: 
len(list(member_pages_coll.find()))

1297

In [19]:
mems = [a['member_ID'] for a in member_pages_coll.find()]

In [20]:
mems_left = set(memberIDs) - set(mems)

In [23]:
mems_left = list(mems_left)

In [24]:
len(mems_left)

43

# Scrape member pages using `scraper` code 

In [21]:
import scraper

In [22]:
scrape1 = scraper.Scraper(browser="Firefox")

Connected to mongodb://localhost:27017/


In [26]:
while mems_left: 
    print mems_left[-1],len(mems_left)
    scrape1.get_member_pages(mems_left[-1]);
    mems_left.pop()

16255072 25
accessing http://allrecipes.com/cook/16255072/about-me/ 

accessing http://allrecipes.com/cook/16255072/favorites/ 

accessing http://allrecipes.com/cook/16255072/following/ 

accessing http://allrecipes.com/cook/16255072/followers/ 

accessing http://allrecipes.com/cook/16255072/reviews/ 

accessing http://allrecipes.com/cook/16255072/made-it/ 

accessing http://allrecipes.com/cook/16255072/recipes/ 

13547633 24
accessing http://allrecipes.com/cook/13547633/about-me/ 

accessing http://allrecipes.com/cook/13547633/favorites/ 

accessing http://allrecipes.com/cook/13547633/following/ 

accessing http://allrecipes.com/cook/13547633/followers/ 

accessing http://allrecipes.com/cook/13547633/reviews/ 

accessing http://allrecipes.com/cook/13547633/made-it/ 

accessing http://allrecipes.com/cook/13547633/recipes/ 

6492551 23
accessing http://allrecipes.com/cook/6492551/about-me/ 

accessing http://allrecipes.com/cook/6492551/favorites/ 

accessing http://allrecipes.com/cook/6

In [28]:
member_pages_coll.count()

1340